In [22]:
%%writefile ../scripts/send.py
#!/usr/bin/env python
import pika
import sys

job_description =''

if len(sys.argv)<2:
    print("Please Specify the job descriptor json file to run as argument to this script.")
    exit(1)
else:
    job_description = open(sys.argv[1],'r').read()

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='tasks')

channel.basic_publish(exchange='',
                      routing_key='tasks',
                      body=job_description)
print(" [x] Sending 'Job Description' to execute.")
print("Details:\n",str(job_description))
print("Success.")
connection.close()

Overwriting ../scripts/send.py


In [5]:
%%writefile ../scripts/receive.py
#!/usr/bin/env python
import pika
import shlex
import subprocess
import re
import pprint

def run_command(command):
    try:
        process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE, universal_newlines=True)
    except FileNotFoundError as e:
        print(e)
        return (-1)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc

def env_str(env_dic):
    env_string=""
    for x,y in env_dic.items():
        env_string =env_string+" --env "+x+"="+y 
    return env_string.strip()

def generate_cmd(job_description):
    command = 'docker run -d '
    req_keys = ['image', 'cmd']
    mem_req = None
    invalid = False
    
    for k in req_keys:
        if k not in job_description.keys():
            print("Job description not valid.")
            print("Missing",k)
            invalid=True
    
    if 'res_req' in job_description.keys():
        if 'memory' in job_description['res_req'] and type(job_description['res_req']['memory'])==int:
            mem_req = job_description['res_req']['memory']
        else:
            print("No memory reqirement mentioned, checking againt difined buffer only.")
            print("Check memory representation must be an number in kb")
            print("Continuing...")
            
    if invalid==True or is_mem_available(mem_req)==False:
        print('Failed to generate Command...')
        if is_mem_available(mem_req)==False:
            print('Not suffuciant Memory Available. Note buffer is 10000kb by default')
        return None
    
    if 'env' in job_description.keys():
        command = command + env_str(job_description['env'])
    command =command+ " -i -t "+job_description['image']+" /bin/bash -c \""+ str(' '.join(e for e in job_description['cmd']))+"\" " 
    return command

def is_mem_available(r_mem_kb, buffer_kb=100000):
    if r_mem_kb==None:
        r_mem_kb = 0
    meminfo = open('/proc/meminfo').read()
    match = re.findall('MemFree:.*',meminfo)[0]
    a_mem_kb = int(match.split()[1])
    if r_mem_kb+buffer_kb < a_mem_kb:
        return True
    else:
        return False

connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.queue_declare(queue='tasks')

def callback(ch, method, properties, body):
    job_desc = eval(body.decode("utf-8"))
    print(" [x] Job description:")
    command = generate_cmd(job_desc)
    print(command)
    if command:
        run_command(command)
#         print()
    print("Finished...")

channel.basic_consume(callback,
                      queue='tasks',
                      no_ack=True)

print(' [*] Waiting for New Job. To exit press CTRL+C')
channel.start_consuming()

Overwriting ../scripts/receive.py


## a docker image
an array of cmd parameters to pass to the image
a dictionary of environment variables to pass to the image
a dictionary of cpu and memory requirements

In [26]:
type(eval(open('../job_desc/job_desc_2.json').read())['res_req']['memory'])

int

In [82]:
job={'image':'ubuntu', 'cmd':['ls','-l'], 'env':{'my_env':'testing', 'your_env':'prod'}, 'res_req':{'cpu':'something', 'memory':'another thing'}}

In [147]:
def run_command(command):
    try:
        process = subprocess.Popen(shlex.split(command), stdout=subprocess.PIPE)
    except FileNotFoundError as e:
        print(e)
        return (-1)
        
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output:
            print (output.strip())
    rc = process.poll()
    return rc

In [148]:
run_command('ls -l ;')

127

In [31]:
import pprint


In [36]:
print = pprint.pprint()

In [38]:
print('sadsd')

'sadsd'
